In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sp
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

import glob
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import string


from nltk.corpus import stopwords
from collections import defaultdict
from gensim import corpora,models, similarities

from scipy import sparse
import numpy as np

In [5]:
directorytemp = glob.glob('sraa/*')
directory = directorytemp[0],directorytemp[2]
documents = []
stemmer = PorterStemmer()

for folder in directory:
    print folder + '....',
    tmp = []
    files = glob.glob(folder+"/*")
    nums = 0
    for fileName in files:
        f = open(fileName,'r')
        data = f.read()
        data = ''.join(i if ord(i)<128 else ' ' for i in data )
        data = data.lower()
        
        #toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
        #dataA = toker.tokenize(data)
        
        dataA = word_tokenize(data)
        dataA_without_punc = [i for i in dataA if i not in string.punctuation]
        
        single = ' '.join([stemmer.stem(w) for w in dataA_without_punc])
        tmp.append(single)
    print 'is created'
    documents.append(tmp)

sraa\realauto.... is created
sraa\simauto.... is created


In [6]:
subDirectory = 'run_sraa'

def preprocess(documents):
    # documents = ['babak is khar','hasan is good']
    stopList = stopwords.words('english')
    print len(stopList)
    texts = [[word for word in document.lower().split() if word not in stopList]#+nontfidf]
         for document in documents]
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    numberOfDocuments = len(documents)
    return [[token for token in text if (frequency[token] > 1 and frequency[token] < numberOfDocuments) ]
        for text in texts]
    
    
    
def makeCorpus(texts):
    # texts = [['babak', 'is' , 'khar'],['hasan', 'is','good']]
    # it means we do some preprocess like: stopword, common word, ...
    dictionary = corpora.Dictionary(texts)
    dictionary.save(subDirectory+'/dictionary_without.dict')
#     print dictionary.token2id
    return [dictionary.doc2bow(text) for text in texts]


numOfDocs = 0
for docs in documents:
    numOfDocs += len(docs)
classes = np.zeros(numOfDocs,dtype=int)

alldocs = []
idx = 0
classLabel = 0
for docs in documents:
    for doc in docs:
        alldocs.append(doc)
        classes[idx] = classLabel
        idx += 1
    classLabel += 1
        
texts = preprocess(alldocs)
corpus = makeCorpus(texts)
corpora.MmCorpus.serialize(subDirectory+'/corpus_without.mm', corpus)
np.savetxt(subDirectory + '/classes_without.dat',classes)


127


In [7]:
from gensim.models.ldamodel import LdaModel

corpus = corpora.MmCorpus(subDirectory+'/corpus_without.mm')
dictionary = corpora.dictionary.Dictionary.load(subDirectory+'/dictionary_without.dict')
classes = np.loadtxt(subDirectory+'/classes_without.dat',dtype=int)


ldaModel = LdaModel(corpus, num_topics=30, id2word = dictionary, passes=20)
ldaModel.save(subDirectory+'/lda_without.model')

tfidfModel = models.TfidfModel(corpus)
tfidfModel.save(subDirectory+'/sraa.tfidf_without_model')

In [11]:
from gensim.models.ldamodel import LdaModel
ldaModel = LdaModel.load(subDirectory+'/lda_without.model')

sparseDataLda = []
ldaFeatureNum = 0
for data in corpus:
    ldaData = ldaModel[data]
    sparseDataLda.append(ldaData)    
    for i in ldaData:
        ldaFeatureNum = max(ldaFeatureNum,i[0])
        
numData = len(corpus)
ldaFeatureNum = 30
dataset = pd.DataFrame(np.zeros((numData,ldaFeatureNum)))

for i in range(numData):
    for t in sparseDataLda[i]:
        dataset[t[0]].ix[i] = t[1]

X1 = dataset[classes== 0]
X2 = dataset[classes== 1]


Y = pd.Series(classes)

Xtrain = pd.concat([X1,X2])
Y1 = Y[Y==0]
Y2 = Y[Y==1]
Ytrain = pd.concat([Y1,Y2])


from sklearn import svm
clf = svm.LinearSVC()
clf.fit(Xtrain,Ytrain)
Ypred = clf.predict(Xtrain)
((Ypred==Ytrain).sum()*1.0)/len(Ytrain)

0.93358181463583767